# tweet類似度検索

自然言語処理（NLP）技術であるBERT（Bidirectional Encoder Representations from Transformers）を用いて、テキストメッセージ同士の類似度を計算し、データベースから最も似たテキストメッセージを検索する。

## tweetを格納するテーブルの作成

MySQLに接続し、必要なテーブルを作成する。

**tweets**

| Column name | Data type | Description |
|-------------|-----------|-------------|
| id          | INT       | テーブルの主キー。自動的にインクリメントされる。 |
| text        | VARCHAR(255) | ツイートの本文。 |
| created_at  | TIMESTAMP | レコードが作成された日時。デフォルトは現在のタイムスタンプ。 |

**tweet_vectors**

| Column name | Data type | Description |
|-------------|-----------|-------------|
| id          | INT       | テーブルの主キー。自動的にインクリメントされる。 |
| tweet_id    | INT       | tweetsテーブルのidを参照する外部キー。 |
| vector      | JSON      | ツイートのBERTによる数値ベクトル表現。 |
| created_at  | TIMESTAMP | レコードが作成された日時。デフォルトは現在のタイムスタンプ。 |

`tweets`にはテキストメッセージを格納する。

`tweet_vectors`には、`tweets`から取得したテキストメッセージをベクトル表現に変換したものを格納する。

In [1]:
import pymysql

# MySQLデータベースへの接続情報を設定
host = "db"  # Docker Composeで定義したMySQLサービスのサービス名
port = 3306  # MySQLのデフォルトのポート番号
user = "user"  # MySQLのユーザ名
password = "password"  # MySQLのパスワード
database = "tweets_db"  # MySQLのデータベース名

def connect_to_database():
    # MySQLデータベースに接続
    return pymysql.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    )

def create_table_if_not_exists(cursor, create_table_query):
    # テーブルを作成するSQL文
    cursor.execute(create_table_query)

    # 変更をコミット（確定）
    connection.commit()

# tweetsテーブルの作成
create_tweets_table_query = """
CREATE TABLE IF NOT EXISTS tweets (
    id INT AUTO_INCREMENT PRIMARY KEY,
    text VARCHAR(255),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

# tweet_vectorsテーブルの作成
create_tweet_vectors_table_query = """
CREATE TABLE IF NOT EXISTS tweet_vectors (
    id INT AUTO_INCREMENT PRIMARY KEY,
    tweet_id INT,
    vector JSON,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (tweet_id) REFERENCES tweets(id)
);
"""

# データベースへの接続
connection = connect_to_database()

# カーソルオブジェクトを作成
cursor = connection.cursor()

# テーブルの作成
create_table_if_not_exists(cursor, create_tweets_table_query)
create_table_if_not_exists(cursor, create_tweet_vectors_table_query)

# MySQLデータベースとの接続を閉じる
connection.close()


## ダミーのtweetの作成と保存

ダミーのtweetを作成してMySQLに保存する。

具体的には、事前に作成したテキストメッセージ`tweets.csv`を読み込み、`tweets`テーブルに保存する。

In [2]:
import configparser
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.sql import insert

# 設定を読み込む
config = configparser.ConfigParser()
config.read('config.ini')

# MySQLの設定
username = config['DATABASE']['USERNAME']
password = config['DATABASE']['PASSWORD']
hostname = config['DATABASE']['HOSTNAME']
database = config['DATABASE']['DATABASE']

engine = create_engine(f"mysql+pymysql://{username}:{password}@{hostname}/{database}")

# ダミーのツイート
df_tweet = pd.read_csv("tweets.csv")

# ツイートをデータベースに保存
with engine.begin() as connection:
    metadata = MetaData()
    metadata.bind = engine

    tweet_table = Table('tweets', metadata, autoload_with=engine)
    for tweet in df_tweet['text']:
        stmt = insert(tweet_table).values(text=tweet)
        connection.execute(stmt)


## BERTによるtweetのベクトル変換

MySQLに保存されているtweetをBERTを利用してベクトル表現に変換し、`tweet_vectors`に保存する。

1. Hugging Faceのtransformersというライブラリを使って、BERTのモデル`bert-base-uncased`とトークナイザをロード
2. `tweets`テーブルからテキストメッセージを取得する
3. テキストメッセージのベクトル変換
  - トークナイザでテキストメッセージをトークン化（モデルに入力できる形式に変換）
  - トークンをBERTモデルに入力してベクトル表現を取得する
4. ベクトル表現の保存

In [3]:
import configparser
import torch
from sqlalchemy import select
from transformers import BertModel, BertTokenizer
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.sql import insert

# 設定を読み込む
config = configparser.ConfigParser()
config.read('config.ini')

# MySQLの設定
username = config['DATABASE']['USERNAME']
password = config['DATABASE']['PASSWORD']
hostname = config['DATABASE']['HOSTNAME']
database = config['DATABASE']['DATABASE']

engine = create_engine(f"mysql+pymysql://{username}:{password}@{hostname}/{database}")

# BERTの設定
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# ツイートをデータベースから取得
with engine.begin() as connection:
    metadata = MetaData()
    metadata.bind = engine
    tweet_table = Table('tweets', metadata, autoload_with=engine)
    s = select(tweet_table.c.id, tweet_table.c.text)
    result = connection.execute(s).fetchall()

    for row in result:
        tweet_id, text = row
        inputs = tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)

        vector = outputs.last_hidden_state[:, 0, :].numpy().tolist()
        # ベクトルをデータベースに保存
        vector_table = Table("tweet_vectors", metadata, autoload_with=engine)
        stmt = insert(vector_table).values(tweet_id=tweet_id, vector=vector)
        connection.execute(stmt)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 任意のtweetと類似するtweetの検索

与えられた入力テキストに最も類似したtweetをデータベースから検索する。

具体的には以下の手順で処理を行う。

1. `tweet_vectors`からすべてのベクトル表現を取得
2. 検証用のテキストメッセージ`verification_tweets.csv`を読み込み、それぞれに次の処理を行う
  - 入力テキストをベクトルに変換
  - 入力テキストのベクトルとデータベース内の各ツイートのベクトルのコサイン類似度を計算
  - 最も類似度が高いツイートを検索し、そのテキストを`tweets`から取得
3. 入力テキストと最も類似すると計算されたテキストを表示

In [4]:
import configparser
import pandas as pd
from sqlalchemy import select
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer
from sqlalchemy import create_engine, Table, MetaData

# 設定を読み込む
config = configparser.ConfigParser()
config.read('config.ini')

# MySQLの設定
username = config['DATABASE']['USERNAME']
password = config['DATABASE']['PASSWORD']
hostname = config['DATABASE']['HOSTNAME']
database = config['DATABASE']['DATABASE']

engine = create_engine(f"mysql+pymysql://{username}:{password}@{hostname}/{database}")

# BERTの設定
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# データベースからベクトルを取得
metadata = MetaData()
metadata.bind = engine
vector_table = Table("tweet_vectors", metadata, autoload_with=engine)
s = select(vector_table.c.tweet_id, vector_table.c.vector)
with engine.begin() as connection:
    result = connection.execute(s).fetchall()

# 'verification_tweets.csv'を読み込む
df = pd.read_csv('verification_tweets.csv')

# DataFrameのtext列を1件ずつ取り出す
for input_text in df['text']:
    # 入力テキストをベクトルに変換
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model(**inputs)
    input_vector = outputs.last_hidden_state[:, 0, :].detach().numpy().tolist()

    max_similarity = 0
    most_similar_tweet = None

    # 各ベクトルと入力テキストのベクトルとの間で類似度を計算
    for row in result:
        tweet_id, vector = row
        similarity = cosine_similarity(input_vector, vector)

        # 最も類似度が高いツイートを見つける
        if similarity[0][0] > max_similarity:
            max_similarity = similarity[0][0]
            most_similar_tweet = tweet_id

    # 最も類似度が高いツイートの本文を取得
    tweet_table = Table("tweets", metadata, autoload_with=engine)
    s = select(tweet_table.c.text).where(tweet_table.c.id == most_similar_tweet)
    with engine.begin() as connection:
        most_similar_tweet_text = connection.execute(s).scalar_one()

    # 最も類似度が高いツイートと入力に使ったテキストを表示
    print(f"Input text: {input_text}")
    print(f"Most similar tweet: {most_similar_tweet_text}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Input text: 一緒に働く田中が冗談を言って、場の雰囲気が良くなった。
Most similar tweet: 犬の喜びそうな顔を見ると、一日の疲れが吹っ飛ぶ。
Input text: 久しぶりに砂浜に行ってリフレッシュできた。
Most similar tweet: 新しいプログラミング言語を学び始めた。チャレンジは成長につながる。
Input text: 昨日読んだ推理小説の最後に驚くべきどんでん返しがあった。
Most similar tweet: 昨日観た映画が素晴らしかった。深いメッセージが心に残った。
Input text: 不具合の修正ばかりで1日が終わったが、やりきると清々しい。
Most similar tweet: 猫の毛づくろいを見ていると癒される。彼らの日常が特別。
Input text: サンスベリアを部屋に置くと有害物質を除去してくれるらしい。
Most similar tweet: パズルゲームの新レベルをクリア。脳を活性化させる感じが好き。
